In [20]:
from openpyxl import load_workbook
import os

def get_sheet_names(file_path):
    workbook = load_workbook(filename=file_path, read_only=True)
    return workbook.sheetnames

# Example: read all Excel files in the current directory and print their sheet names
for file in os.listdir():
    if file.endswith('.xlsx'):
        print(f"File: {file}")
        try:
            sheet_names = get_sheet_names(file)
            print("Sheet names:", sheet_names)
        except Exception as e:
            print(f"Could not read {file}: {e}")

#read an Excel file and extract specific data
import pandas as pd
file_path = 'data_districts.xlsx'

#discard the first 15 rows and read the rest
data = pd.read_excel(file_path, skiprows=15)




File: data_districts.xlsx
Sheet names: ['Bageshwar', 'Almora', 'Chamoli', 'Champawat', 'Dehradun', 'Haridwar', 'Tehri', 'Pauri', 'Uttarkashi', 'Rudraprayag', 'Udham Singh Nagar', 'Nainital', 'Pithoragarh']
File: malaria.xlsx
Sheet names: ['Bageshwar', 'Almora', 'Chamoli', 'Champawat', 'Dehradun', 'Haridwar', 'Tehri', 'Pauri', 'Uttarkashi', 'Rudraprayag', 'Udham Singh Nagar', 'Nainital', 'Pithoragarh']


In [21]:
#strip different sheets for district wise data
#read name of each sheet
sheet_names = pd.ExcelFile(file_path).sheet_names

monthly_means = {}
for sheet in sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet, skiprows=15)
    # Discard columns PRECTOTCORR and T2M_RANGE at the beginning
    df = df.drop(columns=['PRECTOTCORR', 'T2M_RANGE'], errors='ignore')
    # Rename QV2M and RH2M columns
    df = df.rename(columns={'QV2M': 'Specific Humidity', 'RH2M': 'Relative Humidity'})
    df['Date'] = pd.to_datetime(dict(year=df['YEAR'], month=df['MO'], day=df['DY']))
    df['MonthStart'] = df['Date'].dt.to_period('M')
    # Group by MonthStart and take mean for each month
    df_monthly = df.groupby('MonthStart').mean(numeric_only=True).reset_index()
    # Keep only the year from MonthStart and rename to 'Year'
    df_monthly['Year'] = df_monthly['MonthStart'].dt.year
    df_monthly = df_monthly.drop(columns=['MonthStart', 'YEAR', 'DY'], errors='ignore')
    cols = ['Year'] + [col for col in df_monthly.columns if col != 'Year']
    df_monthly = df_monthly[cols]
    month_map = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
                 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}
    if 'MO' in df_monthly.columns:
        df_monthly['MO'] = df_monthly['MO'].map(month_map)
        df_monthly = df_monthly.rename(columns={'MO': 'Month'})
    # Ensure consistent column order and only required columns
    desired_cols = ['Year', 'Month', 'Specific Humidity', 'Relative Humidity', 'T2M', 'T2M_MAX', 'T2M_MIN']
    for col in desired_cols:
        if col not in df_monthly.columns:
            df_monthly[col] = None
    df_monthly = df_monthly[desired_cols]
    monthly_means[sheet] = df_monthly

    # Print the first few rows for each district
    print(f"Monthly mean for district '{sheet}':")
    print(df_monthly.head())
    # Save the dataframe for each district in single file but different sheets of a new Excel file for years 2010-2019 only
    output_file = 'weather_filtered.xlsx'
    df_2010_2019 = df_monthly[(df_monthly['Year'] >= 2010) & (df_monthly['Year'] <= 2019)].copy()
    df_2010_2019 = df_2010_2019[desired_cols]
    with pd.ExcelWriter(output_file, engine='openpyxl', mode='a' if os.path.exists(output_file) else 'w') as writer:
        df_2010_2019.to_excel(writer, sheet_name=sheet, index=False)
    print(f"Saved monthly means for district '{sheet}' (2010-2019) to {output_file}")
    

    #plot the data for each district
    # import matplotlib.pyplot as plt 
    # plt.figure(figsize=(10, 5))
    # plt.plot(df_monthly['Year'], df_monthly['T2M'], marker='o', label='Mean Temperature (K)')
    # plt.title(f'Monthly Means for {sheet}')
    # plt.xlabel('Year')
    # plt.ylabel('Values')
    # plt.legend()
    # plt.grid()
    # #plt.savefig(f'{sheet}_monthly_means.png')
    # plt.close()



Monthly mean for district 'Bageshwar':
   Year     Month  Specific Humidity  Relative Humidity        T2M    T2M_MAX  \
0  1992   January           2.732903          41.462258   4.894839  12.180323   
1  1992  February           2.516897          41.159655   4.012759  11.910345   
2  1992     March           3.046129          34.942258   9.371613  17.176774   
3  1992     April           3.138667          24.820000  14.779667  22.872667   
4  1992       May           4.489355          30.679032  17.393226  25.124839   

    T2M_MIN  
0  0.053548  
1 -1.891379  
2  3.187419  
3  7.374000  
4  9.804516  
Saved monthly means for district 'Bageshwar' (2010-2019) to weather_filtered.xlsx
Monthly mean for district 'Almora':
   Year     Month  Specific Humidity  Relative Humidity        T2M    T2M_MAX  \
0  1992   January           4.047419          45.210968  11.282581  18.973548   
1  1992  February           3.797586          42.668276  11.615517  20.127931   
2  1992     March           4

In [ ]:
import matplotlib.pyplot as plt

for sheet, df_monthly in monthly_means.items():
    df_plot = df_monthly[(df_monthly['Year'] >= 2010) & (df_monthly['Year'] <= 2019)]
    if df_plot.empty:
        continue
    years = df_plot['Year'].unique()
    for year in years:
        df_year = df_plot[df_plot['Year'] == year]
        plt.figure(figsize=(10, 5))
        if 'T2M' in df_year.columns:
            plt.plot(df_year['Month'], df_year['T2M'], marker='o', label='Mean Temperature (K)')
        if 'Specific Humidity' in df_year.columns:
            plt.plot(df_year['Month'], df_year['Specific Humidity'], marker='x', label='Specific Humidity (kg/kg)')
        if 'Relative Humidity' in df_year.columns:
            plt.plot(df_year['Month'], df_year['Relative Humidity'], marker='s', label='Relative Humidity (%)')
        plt.title(f'{sheet} - Monthly Means - {year}')
        plt.xlabel('Month')
        plt.ylabel('Values')
        plt.legend()
        plt.grid()
        plt.xticks(rotation=45)
        plt.tight_layout()
        # save plots in the figs directory
        if not os.path.exists('figs'):
            os.makedirs('figs')
        # plt.savefig(f'figs/{sheet}_monthly_means_{year}.png')
        plt.close()
        # plt.show()